### PyTorch is a Python-based scientific computing package serving two broad purposes:
1. A replacement for NumPy to use the power of GPUs and other accelerators.
2. An automatic differentiation library that is useful to implement neural networks.

1. https://pytorch.org/get-started/locally/
2. https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
3. https://github.com/yunjey/pytorch-tutorial

In [2]:
#Import libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Device configuration -
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Hyper-parameters 
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
hidden_size = 500
learning_rate = 0.001

In [5]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [6]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [9]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [11]:
#Model
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [12]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [13]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3872
Epoch [1/5], Step [200/600], Loss: 0.1517
Epoch [1/5], Step [300/600], Loss: 0.2866
Epoch [1/5], Step [400/600], Loss: 0.1271
Epoch [1/5], Step [500/600], Loss: 0.0745
Epoch [1/5], Step [600/600], Loss: 0.1267
Epoch [2/5], Step [100/600], Loss: 0.1365
Epoch [2/5], Step [200/600], Loss: 0.1405
Epoch [2/5], Step [300/600], Loss: 0.1753
Epoch [2/5], Step [400/600], Loss: 0.0790
Epoch [2/5], Step [500/600], Loss: 0.1145
Epoch [2/5], Step [600/600], Loss: 0.1076
Epoch [3/5], Step [100/600], Loss: 0.0404
Epoch [3/5], Step [200/600], Loss: 0.0746
Epoch [3/5], Step [300/600], Loss: 0.0749
Epoch [3/5], Step [400/600], Loss: 0.0478
Epoch [3/5], Step [500/600], Loss: 0.0609
Epoch [3/5], Step [600/600], Loss: 0.0812
Epoch [4/5], Step [100/600], Loss: 0.0188
Epoch [4/5], Step [200/600], Loss: 0.0800
Epoch [4/5], Step [300/600], Loss: 0.0667
Epoch [4/5], Step [400/600], Loss: 0.0520
Epoch [4/5], Step [500/600], Loss: 0.0849
Epoch [4/5], Step [600/600], Loss:

In [14]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 97.62000274658203 %


In [7]:
# Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')